In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from models import mxresnet

In [4]:
from modules.ranger import Ranger

In [7]:
getattr(globals()['mxresnet'], 'mxresnet18')

functools.partial(<function mxresnet at 0x7f93d7c9a488>, expansion=1, n_layers=[2, 2, 2, 2], name='mxresnet18')

In [8]:
getattr(mxresnet, 'mxresnet18')

functools.partial(<function mxresnet at 0x7f93d7c9a488>, expansion=1, n_layers=[2, 2, 2, 2], name='mxresnet18')

In [11]:
globals()['mxresnet']

<module 'models.mxresnet' from '/home/litemax/kaggle/severstal-steel/models/mxresnet.py'>